In [1]:
#!/usr/bin/env python3
from random import randint
from Crypto.Util.number import *
from hashlib import sha256, md5
from ecdsa import SECP256k1
from ecdsa.ecdsa import Public_key, Private_key, Signature

FLAG = "flag"

E = SECP256k1
G, n = E.generator, E.order

d = randint(1, n)
print(d)

pubkey = Public_key(G, d*G)
prikey = Private_key(pubkey, d)
print(f'P = ({pubkey.point.x()}, {pubkey.point.y()})')

33717798343342530938757884144696676884309003060150418710908329730043516527662
P = (91798623758206733223317495906355959263149226476252400210291726984941720890246, 11219465136598658418271245518928073074098395723828802160539163549684279529788)


In [2]:
def getPair(msg):
    h = sha256(msg.encode()).digest()
    k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(prikey.secret_multiplier) + h).hexdigest(), 16)
    print(k)
    sig = prikey.sign(bytes_to_long(h), k)
    return (bytes_to_long(h), k-(int(md5(b'secret').hexdigest(), 16) << 128), sig.r, sig.s)

In [3]:
md5(b'secret').hexdigest()

'5ebe2294ecd0e0f08eab7690d2a6ee69'

In [4]:
h1 = sha256("A".encode()).digest()
h2 = sha256("B".encode()).digest()

In [5]:
h1k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(prikey.secret_multiplier) + h1).hexdigest(), 16)
h2k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(prikey.secret_multiplier) + h2).hexdigest(), 16)
print(md5(long_to_bytes(prikey.secret_multiplier) + h1).hexdigest())
print(h1k)
print(h2k)

474574e658d31f988242f0cbb9a085a1
42853347383522459682061542032602724326641508808746299105300629223919363065249
42853347383522459682061542032602724326806465361686063242736592014720884595188


In [6]:
42853347383522459682061542032602724326711805998486029767783327103096810254910 - int(md5(long_to_bytes(prikey.secret_multiplier) + h1).hexdigest(), 16)

42853347383522459682061542032602724326617070171292580522407351922876257774749

In [7]:
42853347383522459682061542032602724326671843698816841687549954081255772857811 - int(md5(long_to_bytes(prikey.secret_multiplier) + h2).hexdigest(), 16)

42853347383522459682061542032602724326412151318683628304738016110233698847711

In [8]:
baseK = 42853347383522459682061542032602724326546772981552849859924654043698810585088
print(baseK)
print(int(md5(b'secret').hexdigest(), 16) << 128)

print(h1k - baseK)
print(h2k - baseK)
print(len(bin(h1k-baseK)[2:]))

42853347383522459682061542032602724326546772981552849859924654043698810585088
42853347383522459682061542032602724326546772981552849859924654043698810585088
94735827193449245375975180220552480161
259692380133213382811937971022074010100
127


In [17]:
# Solve
# r, s

# h1, _k1, r1, s1 = getPair("A")
# h2, _k2, r2, s2 = getPair("B")

h1 = bytes_to_long(sha256("A".encode()).digest())
r1 = 16357817616048414572510173075729217041394456493509961448505749657426247777843
s1 = 51748112016471053220430514369599663961770787986932028467346228728169657624151

h2 = bytes_to_long(sha256("B".encode()).digest())
r2 = 52850301281162051958004179951224632024102789840174794535530526838028580351029
s2 = 33951670704847104798562491835349038883273418841901845398791994869986677301265

t = (-1* inverse(s1, n)*s2*r1*inverse(r2, n)) %n
u = (inverse(s1, n)*r1*h2*inverse(r2, n) - inverse(s1, n) * h1 + (t+1)*baseK) %n
K = 2^128

# h1, _k1, r1, s1, h2, _k2, r2, s2
h1, r1, s1, h2, r2, s2

(38720307207599648528211736436817930416103789439318178660273974026535871438845,
 16357817616048414572510173075729217041394456493509961448505749657426247777843,
 51748112016471053220430514369599663961770787986932028467346228728169657624151,
 101089167133868482642301738280228084727114034694682239136375376240207457290844,
 52850301281162051958004179951224632024102789840174794535530526838028580351029,
 33951670704847104798562491835349038883273418841901845398791994869986677301265)

In [18]:
print(K)
B = matrix(ZZ, [[n, 0, 0], [t, 1, 0], [u, 0, K]])
B.LLL()

340282366920938463463374607431768211456


[ 187378089026535655899655420624973157486  -39675609313070595445567472931648011033                                        0]
[ -57629297996773195343421478361731524043  101001157603293355524690901802694853988  340282366920938463463374607431768211456]
[-137064426525898910338553240005831421117 -588937453536132064209118821432104827466                                        0]

In [19]:
dn = ((s1 * (57629297996773195343421478361731524043 + baseK)  - h1) * inverse(r1, n)) %n
print(dn)

d2 = ((s2 * (101001157603293355524690901802694853988 + baseK)  - h2) * inverse(r2, n)) %n
print(d2)

19972761981093915324939593305051321935639939008326997602327308879666237417259
19972761981093915324939593305051321935639939008326997602327308879666237417259


In [23]:
print(dn)
dn = 37913751414115536947901157134231041236627920528277356985630767768138612399437
pubkey = Public_key(G, dn*G)
prikey = Private_key(pubkey, d)
print(f'P = ({pubkey.point.x()}, {pubkey.point.y()})')

h = sha256("Kuruwa".encode()).digest()
k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(prikey.secret_multiplier) + h).hexdigest(), 16)
sig = prikey.sign(bytes_to_long(h), k)
print(f'({sig.r}, {sig.s})')

hn = bytes_to_long(sha256("Kuruwa".encode()).digest())
pubkey.verifies(hn, Signature(int(52624182320304646480692235524456092742639394953852302530179208738281723858475), int(23266584372881447466615630199376157791356863981137456151023603638440639879421)))

86686489963079473003923631694691783383916659017731398910009146211606071635471
P = (92145270807322215407853536224656612777575935234901879683701560206757423236709, 49450944643101082506504531140571806838183271192493825100681232498822262677240)
(52624182320304646480692235524456092742639394953852302530179208738281723858475, 23266584372881447466615630199376157791356863981137456151023603638440639879421)


False